# Risk Analysis
Author: Sarah Hülsen

Code to replicate main results of *Mangroves and their services are at risk from tropical cyclones and sea level rise under climate change*.
This notebook contains code to calculate:
* the combined risk index
* global mangrove areas at risk
* Dabalà et al. priority areas at risk
* the area-weighted risk index for the planning units by Dabalà

# Calculate composite risk index

In [ ]:
from pathlib import Path
import numpy as np
import geopandas as gpd 
import pandas as pd


In [ ]:
# define data path
data_dir = Path("../../data/")
fig_dir = Path("../../figures")

In [ ]:
# define variables for analysis
models = ['miroc6', 'cesm2', 'ecearth6', 'median']
ssps = ['245', '370', '585']
resolution = '0150'
mollweide = '+proj=moll +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +type=crs'
proj_eck4 = '+proj=eck4 +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs'

## Combined risk index computation

In [ ]:
threshold = 1 # doubling in frequency

for ssp in ssps:
    df = pd.read_csv(data_dir.joinpath(f'Bunting_mangroves_TC_global_{resolution}as_MIT_median_ssp{ssp}_2cal_changemap_analysed.csv'))
    df[f'risk'] = 0
    
    for cat in range(1, 4):
        # define risk per TC intensity category
        df[f'TC{cat}_risk'] = np.where((df[f'TC{cat}_ratio'] >= threshold) | (df[f'TC{cat}_new_affected'] ==1), 1, 0) # convert TC data to risk indicator
        df[f'risk'] = np.where(df[f'RSLR{cat}_ssp{ssp}'] == 1, cat, df[f'risk']) # RSLR information to risk column
    
    # keep only highest risk category
    df['TC1_risk'] = np.where((df['TC2_risk']==1) | (df['TC3_risk'] ==1), 0, df['TC1_risk'])
    df['TC2_risk'] = np.where(df['TC3_risk']==1, 0, df['TC2_risk'])
   
    # adjust risk index with TC information
    df['risk'] = np.where(df['TC2_risk'] == 1, (df[f'risk'] + 1), df['risk'])
    df['risk'] = np.where(df['TC3_risk'] == 1, (df[f'risk'] + 2), df['risk'])
    df['risk'] = np.where((df['TC1_risk']==0) & (df['TC2_risk']==0) & (df['TC3_risk'] ==0), 0, df['risk'])
    
    # save
    df.to_csv(data_dir.joinpath(f'Bunting_mangroves_TC_global_{resolution}as_MIT_median_ssp{ssp}_2cal_risk.csv'), index=False)
        

## Mangrove areal extent at risk statistics

In [ ]:
cols = ['area', 'risk']
df_list = []
for ssp in ssps:
    df = pd.read_csv(data_dir.joinpath(f'Bunting_mangroves_TC_global_{resolution}as_MIT_median_ssp{ssp}_2cal_risk.csv'))
    df = df[cols]
    total_area = df['area'].sum()
    df = df.groupby('risk').sum()
    df[f'SSP{ssp}'] = df['area']/total_area*100
    df = df.drop(columns='area')
    df = df.transpose()
    df_list.append(df)
dataframes = pd.concat(df_list)
dataframes = dataframes.fillna(0)
#dataframes = dataframes.round(2)
dataframes.to_csv(data_dir.joinpath(f'Bunting_mangroves_TC_global_{resolution}as_MIT_median_2cal_risk_area_percentages.csv'), index='risk')


In [ ]:
dataframes

risk,0,1,2,3,4,5
SSP245,50.509169,0.002316,9.242950,24.353509,14.621361,1.270695
SSP370,52.701139,0.000000,1.302925,9.602359,23.506331,12.887247
SSP585,44.437913,0.000000,0.001729,9.579851,26.482559,19.497949


## Priority areas at risk 

### Merge risk index with mangrove priority area data

In [ ]:
# load priority area file
cols = ['ID', 'geometry', 'rank', 'AreaGMWKm', 'Fishing_Intensity',
       'soil_carbon', 'biomass_carbon', 'InvFish', 'InvProperty', 'InvPeople',
       'InvCarbon', 'TOT_STOCK', 'POP', 'InvPropertyCost', 'InvPeopleCost',
       'InvFishCost', 'Tot_Carbon', 'InvCarbonCost', 'Fishing_Intensity_prc',
       'TOT_STOCK_prc', 'POP_prc', 'Tot_Carbon_prc', 'country', 'continent']

pa_df = pd.read_csv(data_dir.joinpath('result_BioServ_gdf.csv'))
pa_df = pa_df[cols]
pa_df['geometry'] = gpd.GeoSeries.from_wkt(pa_df['geometry'])
pa_gdf = gpd.GeoDataFrame(pa_df, geometry='geometry', crs=mollweide)

for ssp in ssps:
    df = pd.read_csv(data_dir.joinpath(f'Bunting_mangroves_TC_global_0150as_MIT_median_ssp{ssp}_2cal_risk.csv'))
    df = df.drop(columns=['index_right'])
    df['geometry'] = gpd.GeoSeries.from_wkt(df['geometry'])
    gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:4326")
    gdf = gdf.to_crs(mollweide)
    join = gpd.sjoin_nearest(gdf, pa_gdf, how='inner', max_distance=100)
    join = join.to_crs(epsg=4326)
    join.to_csv(data_dir.joinpath(f'Bunting_mangroves_TC_global_0150as_MIT_median_ssp{ssp}_2cal_risk_BioServ.csv'), index=False)

In [ ]:
# calculate priority ranks for 25th and 10th percentile priority areas
df = pd.read_csv(data_dir.joinpath('result_BioServ_gdf.csv'))
data = df['rank']
p25 = np.percentile(data, 25)
p10 = np.percentile(data, 10)
print(p25)
print(p10)

8.0
2.0


In [ ]:
cols = ['area', 'risk']
df_list = []
for ssp in ssps:
    df = pd.read_csv(data_dir.joinpath(f'Bunting_mangroves_TC_global_0150as_MIT_median_ssp{ssp}_2cal_risk_BioServ.csv'))
    df = df[df['rank']<=p25]
    total_area = df['area'].sum()
    df = df[cols]
    df = df.groupby('risk').sum()
    df[f'SSP{ssp}'] = df['area']/total_area*100
    df = df.drop(columns='area')
    df = df.transpose()
    df_list.append(df)
dataframes = pd.concat(df_list)
dataframes = dataframes.fillna(0)
#dataframes = dataframes.round(2)
dataframes.to_csv(data_dir.joinpath(f'Bunting_mangroves_TC_global_0150as_MIT_median_2cal_risk_BioServ_area_percentages_p25.csv'), index= 'risk')

In [ ]:
dataframes

risk,0,1,2,3,4,5
SSP245,31.928173,0.017309,25.347788,26.960157,15.527844,0.218729
SSP370,37.828239,0.000000,0.928897,17.089379,29.030091,15.123394
SSP585,28.566080,0.000000,0.017309,20.250120,28.451217,22.715273


In [ ]:
cols = ['area', 'risk']
df_list = []
for ssp in ssps:
    df = pd.read_csv(data_dir.joinpath(f'Bunting_mangroves_TC_global_0150as_MIT_median_ssp{ssp}_2cal_risk_BioServ.csv'))
    df = df[df['rank']<=p10]
    total_area = df['area'].sum()
    df = df[cols]
    df = df.groupby('risk').sum()
    df[f'SSP{ssp}'] = df['area']/total_area*100
    df = df.drop(columns='area')
    df = df.transpose()
    df_list.append(df)
dataframes = pd.concat(df_list)
dataframes = dataframes.fillna(0)
#dataframes = dataframes.round(2)
dataframes.to_csv(data_dir.joinpath(f'Bunting_mangroves_TC_global_0150as_MIT_median_2cal_risk_BioServ_area_percentages_p10.csv'), index= 'risk')

In [ ]:
dataframes

risk,0,1,2,3,4,5
SSP245,29.067679,0.022468,11.401565,34.694614,23.997368,0.816306
SSP370,30.203163,0.000000,0.023574,11.357574,33.440037,24.975652
SSP585,26.273860,0.000000,0.022468,3.165306,36.188620,34.349746


## Area-weighted risk scores per priority unit

In [ ]:
cols = ['ID', 'risk', 'area']
dfs = []

# compute average risk score per PA ID (across all mangrove centroids associated with ID)
for ssp in ssps:
    df = pd.read_csv(data_dir.joinpath(f'Bunting_mangroves_TC_global_0150as_MIT_median_ssp{ssp}_2cal_risk_BioServ.csv'))
    df = df[cols]
    df = df.groupby(['ID', 'risk'], as_index=False)[['area']].sum()
    
    result = df.groupby('ID').apply(lambda x: pd.Series({
        f'ssp{ssp}_weighted_risk': (x['risk'] * x['area']).sum() / x['area'].sum(),
        'area': x['area'].sum()
    })).reset_index()
    
    result[f'ssp{ssp}_weighted_risk'] = result[f'ssp{ssp}_weighted_risk'].astype('int')
    dfs.append(result)

final_df = dfs[0]
for i, df in enumerate(dfs[1:], start=1):
    ssp = ssps[i]
    final_df = final_df.merge(df[['ID', f'ssp{ssp}_weighted_risk']], on='ID', how='outer')

final_df = final_df.loc[:, ~final_df.columns.duplicated()]
final_df.to_csv(data_dir.joinpath('PA_risk_area_weighted_all_ssps.csv'), index=False)

# merge back with PA data
cols = ['ID', 'geometry', 'rank', 'AreaGMWKm', 'Fishing_Intensity',
       'soil_carbon', 'biomass_carbon', 'InvFish', 'InvProperty', 'InvPeople',
       'InvCarbon', 'TOT_STOCK', 'POP', 'InvPropertyCost', 'InvPeopleCost',
       'InvFishCost', 'Tot_Carbon', 'InvCarbonCost', 'Fishing_Intensity_prc',
       'TOT_STOCK_prc', 'POP_prc', 'Tot_Carbon_prc', 'country', 'continent']

pa_df = pd.read_csv(data_dir.joinpath('gdf', 'result_BioServ_gdf.csv'))
pa_df = pa_df[cols]
combined_df = final_df.merge(pa_df, on='ID', how='left')
combined_df.to_csv(data_dir.joinpath(f'Bunting_mangroves_TC_global_0150as_MIT_median_BioServ_area_weighted_risk_PUs_all_ssps.csv'), index= 'risk')